In [1]:
import torch
import numpy as np

from datasets.dataset import transform_dataset, kfold_dataset
from DRNet import train, DRNet

In [2]:
# Read datasets
name = 'adult'
X, Y, X_headers, Y_headers = transform_dataset(name, method='onehot-compare', negations=False, labels='binary')
datasets = kfold_dataset(X, Y, shuffle=1)
X_train, X_test, Y_train, Y_test = datasets[0]

train_set = torch.utils.data.TensorDataset(torch.Tensor(X_train.to_numpy()), torch.Tensor(Y_train))
test_set = torch.utils.data.TensorDataset(torch.Tensor(X_test.to_numpy()), torch.Tensor(Y_test))

In [4]:
# Train DR-Net
# Default learning rate (1e-2), and_lam (1e-2), and and_lam (1e-5) usually work the best. A large epochs number is necessary for a sparse rule set i.e 10000 epochs.
net = DRNet(train_set[:][0].size(1), 50, 1)
train(net, train_set, test_set=test_set, device='cuda', lr=1e-2, epochs=2000, batch_size=400,
      and_lam=1e-2, or_lam=1e-5, num_alter=500)

Epoch: 100%|██████████| 2000/2000 [07:36<00:00,  4.38it/s, loss=0.544, epoch accu=0.831, test accu=0.836, num rules=11, sparsity=0.907]                                                                                                                                                 


In [5]:
# Get accuracy and the rule net
accu = (net.predict(np.array(X_test)) == Y_test).mean()
rules = net.get_rules(X_headers)
print(f'Accuracy: {accu}, num rules: {len(rules)}, num conditions: {sum(map(len, rules))}')

Accuracy: 0.836399801093983, num rules: 11, num conditions: 131
